In [25]:
#importing the necessary libraries 

import numpy as np
import pandas as pd
import os 
from PIL import Image
import matplotlib.pyplot as plt

import random
import tensorflow as tf
from keras.models import Sequential
from keras.layers import MaxPooling2D,Conv2D,Dense,Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers

In [26]:
# setting up directories

base_dir = '../input/state-farm-distracted-driver-detection/' # base directory
images_dir = os.path.join(base_dir,'imgs/')                   # images directory
test_dir = os.path.join(base_dir,'imgs/test/')                # test directory
train_dir = os.path.join(base_dir,'imgs/train/')              # train directory

imgs_list = pd.read_csv(os.path.join(base_dir,'driver_imgs_list.csv'))  # images list csv 
sample_sub = pd.read_csv(os.path.join(base_dir,'sample_submission.csv'))       # sample submission

In [27]:
len(imgs_list)

All the images are in jpg format, we will have to

In [28]:
# checking the classes avaliable 
classes = imgs_list['classname'].unique()
classes

This is each class desc given in question

*    c0: safe driving
*    c1: texting - right
*    c2: talking on the phone - right
*    c3: texting - left
*    c4: talking on the phone - left
*    c5: operating the radio
*    c6: drinking
*    c7: reaching behind
*    c8: hair and makeup
*    c9: talking to passenger


In [29]:
#adding class desfination according to the given question

class_def = {
    
    'c0': 'safe driving',
    'c1': 'texting - right',
    'c2': 'talking on the phone - right',
    'c3': 'texting - left',
    'c4': 'talking on the phone - left',
    'c5': 'operating the radio',
    'c6': 'drinking',
    'c7': 'reaching behind',
    'c8': 'hair and makeup',
    'c9': 'talking to passenger'

}


In [35]:
#checking class distribution

imgs_list.groupby('classname')['img'].count().sort_values().plot(kind='bar')
plt.ylabel('Number of images')
plt.show()

In [30]:
imgs_list.head()

In [31]:
#randomly printing 30 images
#since all the images are in jpg format
#using PIL Image library for printing the images

fig = plt.figure(figsize=(30,20)) # image size               

for i in range(30):
    random_img = random.randint(0,imgs_list.shape[0])
    img = Image.open(os.path.join(base_dir,'imgs/train/')+str(imgs_list.loc[random_img,'classname']+'/')
                    + str(imgs_list.loc[random_img,'img']))
    fig.add_subplot(6,5,i+1)
    plt.imshow(img)
    plt.title(class_def[imgs_list.loc[random_img,'classname']])
    plt.axis('off')

plt.tight_layout()
plt.show()


In [34]:
# size of each image

img = Image.open('../input/state-farm-distracted-driver-detection/imgs/test/img_1.jpg')
print(img.size)

Now we have analysed the data, moving onto generation matrices of every image

In [37]:
# creating the train data generator and test data generator
 
image_size = (128,128)  # image shape
batch_size = 32
val_size = 0.2

# initialising image generator
train_data_gen = ImageDataGenerator(rescale=1./127,validation_split= val_size)
test_data_gen = ImageDataGenerator(rescale=1./127)

In [42]:
# after initialising image generator, extract the images from the directories defined above


train_generator = train_data_gen.flow_from_directory(train_dir,
                                                     target_size = image_size,
                                                     batch_size = batch_size,
                                                     seed=42, 
                                                     shuffle=True,
                                                     subset='training')

val_generator =  train_data_gen.flow_from_directory(train_dir,
                                               target_size = image_size,
                                               batch_size = batch_size,
                                               seed=42, 
                                               shuffle=True,
                                               subset='validation')

In [43]:
# check the data for one batch

for data_batch,label_batch in train_generator:
    print(data_batch.shape)   # train batch
    print(label_batch.shape)  # label batch
    break

* 32 images in each batch
* 128X128: Each image size
* 3 represents RGB layer

Formulated the matrices, now building the model

In [67]:
model1 = Sequential()                                                  # creating a sequential model
model1.add(Conv2D(32,(3,3),activation='relu',input_shape=(128,128,3))) # taking units of 32 and filter of 3x3
model1.add(MaxPooling2D(2,2))                                          # maxpool layer with 2x2 filter   
model1.add(Conv2D(64,(3,3),activation='relu'))                         # taking units of 64 and filter of 3x3
model1.add(MaxPooling2D(2,2))
model1.add(Conv2D(128,(3,3),activation='relu'))                        # taking units of 128 and filter of 3x3
model1.add(MaxPooling2D(2,2))
model1.add(Conv2D(256,(3,3),activation='relu'))                        # taking units of 256 and filter of 3x3
model1.add(MaxPooling2D(2,2))
model1.add(Flatten())                                                  # flattening the data to feed into to Dense layer
model1.add(Dense(4096,activation='relu'))                              # taking units of 4096
model1.add(Dense(2048,activation='relu'))                              # taking units of 2048
model1.add(Dense(1024,activation='relu'))                              # taking units of 1024
model1.add(Dense(512,activation='relu'))                               # taking units of 512 
model1.add(Dense(128,activation='relu'))                               # taking units of 128
model1.add(Dense(10,activation='softmax'))                             # output later with units of 10 since 10 labels

model1.summary() # to print summary of model architecture

In [60]:
model2 = Sequential()                                                  # creating a sequential model
model2.add(Conv2D(32,(3,3),activation='relu',input_shape=(128,128,3))) # taking units of 32 and filter of 3x3
model2.add(MaxPooling2D(2,2))                                          # maxpool layer with 2x2 filter   
model2.add(Conv2D(64,(3,3),activation='relu'))                         # taking units of 64 and filter of 3x3
model2.add(MaxPooling2D(2,2))
model2.add(Conv2D(128,(3,3),activation='relu'))                        # taking units of 128 and filter of 3x3
model2.add(MaxPooling2D(2,2))
model2.add(Conv2D(256,(3,3),activation='relu'))                        # taking units of 256 and filter of 3x3
model2.add(MaxPooling2D(2,2))
model2.add(Conv2D(512,(3,3),activation='relu'))                        # taking units of 512 and filter of 3x3
model2.add(MaxPooling2D(2,2))    
model2.add(Flatten())                                                  # flattening the data to feed into to Dense layer
model2.add(Dense(1024,activation='relu'))                              # taking units of 1024
model2.add(Dense(512,activation='relu'))                               # taking units of 512 
model2.add(Dense(128,activation='relu'))                               # taking units of 128
model2.add(Dense(10,activation='softmax'))                             # output later with units of 10 since 10 labels

model2.summary() # to print summary of model architecture

In [71]:
model3 = Sequential()                                                  # creating a sequential model
model3.add(Conv2D(32,(2,2),activation='relu',input_shape=(128,128,3))) # taking units of 32 and filter of 3x3
model3.add(MaxPooling2D(2,2))                                          # maxpool layer with 2x2 filter   
model3.add(Conv2D(64,(2,2),activation='relu'))                         # taking units of 64 and filter of 3x3
model3.add(MaxPooling2D(2,2))
model3.add(Conv2D(128,(2,2),activation='relu'))                        # taking units of 128 and filter of 3x3
model3.add(MaxPooling2D(2,2))
model3.add(Conv2D(256,(2,2),activation='relu'))                        # taking units of 256 and filter of 3x3
model3.add(MaxPooling2D(2,2))
model3.add(Conv2D(512,(2,2),activation='relu'))                        # taking units of 512 and filter of 3x3
model3.add(MaxPooling2D(2,2)) 
model3.add(Flatten())                                                  # flattening the data to feed into to Dense layer
model3.add(Dense(2048,activation='relu'))                              # taking units of 1024
model3.add(Dense(1024,activation='relu'))                              # taking units of 1024
model3.add(Dense(512,activation='relu'))                               # taking units of 512 
model3.add(Dense(128,activation='relu'))                               # taking units of 128
model3.add(Dense(10,activation='softmax'))                             # output later with units of 10 since 10 labels

model3.summary() # to print summary of model architecture



In [73]:
model1.compile('adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [75]:
epochs = 10

history = model1.fit(train_generator,
         steps_per_epoch=240,
         epochs=epochs,
         validation_data=val_generator,
         validation_steps=60
        )

In [76]:
model2.compile('adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [77]:
epochs = 10

history2 = model2.fit(train_generator,
         steps_per_epoch=240,
         epochs=epochs,
         validation_data=val_generator,
         validation_steps=60
        )

In [79]:
model3.compile('adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [80]:
epochs = 10

history3 = model3.fit(train_generator,
         steps_per_epoch=240,
         epochs=epochs,
         validation_data=val_generator,
         validation_steps=60
        )

Comparing model 1,2 and 3. Model 1 gave the best accuracy

In [95]:
tr_loss = history.history['loss']
tr_accuracy = history.history['accuracy']

val_loss = history.history['val_loss']
val_accuracy = history.history['val_accuracy']

In [96]:
epchs = list(range(1,len(tr_loss)+1))
plt.plot(epchs,tr_loss,label='Train')
plt.plot(epchs,val_loss,label='Test')
plt.title("Training and Validation loss")
plt.legend()
plt.show()


plt.plot(epchs,tr_accuracy,label='Train')
plt.plot(epchs,val_accuracy,label='Test')
plt.title("Training and Validation accuracy")
plt.legend()
plt.show()

In [97]:
test_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

test_data = test_gen.flow_from_directory(
    images_dir,
    shuffle = False,
    target_size = image_size,
    classes = ['test'],
    batch_size = 32
)

In [98]:
preds = model1.predict(test_data)